<a href="https://colab.research.google.com/github/waynelee9511cloud/my-colab-notebooks/blob/main/Training_code_for_med_gemma_4B_1028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
MedGemma 醫療術語校正模型訓練程式 (修復 OOM 版本)
用途：訓練模型識別醫療術語是否正確，如果錯誤則提供修正建議
支援：自動讀取並合併 Excel 中的所有工作表 (MH, CM, AE)
改進：
  1. 資料過採樣（錯誤比例提升至 40%）
  2. 優化記憶體使用（更小的 batch size 和序列長度）
  3. 完整的 F1 Score 計算
  4. 訓練後立即測試
修復：
  - Batch size: 4 → 2（減少記憶體）
  - Max length: 512 → 384（減少記憶體）
  - Gradient accumulation: 4 → 8（保持有效 batch size = 16）
  - 限制過採樣上限（避免資料量過大）
"""

# ============================================
# 第一部分：安裝必要的套件
# ============================================

!pip install -q transformers datasets accelerate bitsandbytes peft openpyxl scikit-learn

# ============================================
# 第二部分：導入所需的函式庫
# ============================================

import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.utils import resample
from google.colab import files, drive
import os
import json
from typing import Dict, List
import re
import numpy as np
import gc

# ============================================
# 第三部分：掛載 Google Drive
# ============================================

print("📁 正在掛載 Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive 掛載完成！")

# ============================================
# 第四部分：上傳 Excel 檔案
# ============================================

print("\n📤 請上傳您的 Excel 訓練檔案")
print("提示：程式會自動讀取 Excel 中的所有工作表")
print("  - 支援多個工作表 (例如: MH, CM, AE)")
print("  - 自動合併所有工作表的資料")
uploaded = files.upload()

excel_file = list(uploaded.keys())[0]
print(f"✅ 已上傳檔案: {excel_file}")

# ============================================
# 第五部分：讀取所有工作表並合併
# ============================================

print("\n📊 正在讀取 Excel 檔案...")

# 讀取 Excel 檔案，取得所有工作表名稱
excel_data = pd.ExcelFile(excel_file)
sheet_names = excel_data.sheet_names

print(f"✅ 發現 {len(sheet_names)} 個工作表: {sheet_names}")

# 讀取每個工作表並合併
all_dataframes = []
sheet_stats = []

for sheet_name in sheet_names:
    df_sheet = pd.read_excel(excel_data, sheet_name=sheet_name)

    # 檢查必要欄位
    required_columns = ['Term_To_Check', 'Correct_Output', 'category']
    missing_columns = [col for col in required_columns if col not in df_sheet.columns]

    if missing_columns:
        print(f"⚠️ 工作表 '{sheet_name}' 缺少欄位: {missing_columns}")
        continue

    # 統計資料
    total = len(df_sheet)
    correct = len(df_sheet[df_sheet['Correct_Output'] == 'No issues found.'])
    error = len(df_sheet[df_sheet['Correct_Output'] != 'No issues found.'])

    sheet_stats.append({
        'sheet': sheet_name,
        'total': total,
        'correct': correct,
        'error': error,
        'error_ratio': error / total if total > 0 else 0
    })

    print(f"  📄 {sheet_name}: {total} 筆 (錯誤: {error} / {error/total*100:.1f}%)")

    all_dataframes.append(df_sheet)

# 合併所有資料
df = pd.concat(all_dataframes, ignore_index=True)

print(f"\n✅ 資料合併完成")
print(f"  總筆數: {len(df)}")
print(f"  正確術語: {len(df[df['Correct_Output'] == 'No issues found.'])} 筆")
print(f"  錯誤術語: {len(df[df['Correct_Output'] != 'No issues found.'])} 筆")

# ============================================
# ⭐ 新增：資料平衡處理（過採樣 + 記憶體優化）
# ============================================

print("\n" + "=" * 80)
print("📊 資料平衡處理（過採樣 + 記憶體優化）")
print("=" * 80)

# ⭐ 降低目標錯誤比例和設定上限，避免資料量過大
TARGET_ERROR_RATIO = 0.40  # 從 0.45 降到 0.40
MAX_TOTAL_SAMPLES = 3000   # ⭐ 新增：限制總樣本數上限

# 計算當前比例
current_error_count = len(df[df['Correct_Output'] != 'No issues found.'])
current_correct_count = len(df[df['Correct_Output'] == 'No issues found.'])
current_ratio = current_error_count / len(df)

print(f"\n📈 原始資料分布:")
print(f"  總筆數: {len(df)}")
print(f"  正確術語: {current_correct_count} ({current_correct_count/len(df)*100:.1f}%)")
print(f"  錯誤術語: {current_error_count} ({current_ratio*100:.1f}%)")
print(f"\n🎯 目標錯誤比例: {TARGET_ERROR_RATIO*100:.1f}%")
print(f"🎯 最大樣本數: {MAX_TOTAL_SAMPLES}")

if current_ratio < TARGET_ERROR_RATIO:
    print(f"\n⚠️ 錯誤比例不足 ({current_ratio*100:.1f}% < {TARGET_ERROR_RATIO*100:.1f}%)")
    print(f"🔄 開始過採樣...")

    # 分離正確和錯誤資料
    df_correct = df[df['Correct_Output'] == 'No issues found.'].copy()
    df_error = df[df['Correct_Output'] != 'No issues found.'].copy()

    # 計算需要的錯誤案例數量
    needed_error_count = int(TARGET_ERROR_RATIO * len(df_correct) / (1 - TARGET_ERROR_RATIO))

    # ⭐ 檢查總樣本數是否會超過上限
    potential_total = len(df_correct) + needed_error_count

    if potential_total > MAX_TOTAL_SAMPLES:
        print(f"\n⚠️ 預計總樣本數 ({potential_total}) 超過上限 ({MAX_TOTAL_SAMPLES})")
        print(f"🔧 調整策略：同時減少正確和過採樣錯誤案例")

        # 重新計算：在上限內達到目標比例
        target_error_count = int(MAX_TOTAL_SAMPLES * TARGET_ERROR_RATIO)
        target_correct_count = MAX_TOTAL_SAMPLES - target_error_count

        # 如果正確案例太多，隨機採樣
        if len(df_correct) > target_correct_count:
            df_correct = resample(
                df_correct,
                replace=False,
                n_samples=target_correct_count,
                random_state=42
            )
            print(f"  正確案例採樣: {len(df_correct)}")

        # 過採樣錯誤案例
        df_error_upsampled = resample(
            df_error,
            replace=True,
            n_samples=target_error_count,
            random_state=42
        )
        print(f"  錯誤案例過採樣: {len(df_error_upsampled)}")

    else:
        # 正常過採樣（不超過上限）
        print(f"\n📊 計算結果:")
        print(f"  原始錯誤案例: {len(df_error)}")
        print(f"  需要錯誤案例: {needed_error_count}")
        print(f"  需要增加: {needed_error_count - len(df_error)}")

        df_error_upsampled = resample(
            df_error,
            replace=True,
            n_samples=needed_error_count,
            random_state=42
        )
        print(f"\n✅ 過採樣完成")
        print(f"  過採樣後錯誤案例: {len(df_error_upsampled)}")

    # 合併資料
    df = pd.concat([df_correct, df_error_upsampled], ignore_index=True)

    # 打亂順序
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # 顯示最終結果
    final_error_count = len(df[df['Correct_Output'] != 'No issues found.'])
    final_correct_count = len(df[df['Correct_Output'] == 'No issues found.'])

    print(f"\n🎉 資料平衡完成！")
    print(f"  總筆數: {len(df)}")
    print(f"  正確術語: {final_correct_count} ({final_correct_count/len(df)*100:.1f}%)")
    print(f"  錯誤術語: {final_error_count} ({final_error_count/len(df)*100:.1f}%)")

    # ⭐ 釋放記憶體
    del df_correct, df_error, df_error_upsampled
    gc.collect()

else:
    print(f"\n✅ 錯誤比例已達標 ({current_ratio*100:.1f}% >= {TARGET_ERROR_RATIO*100:.1f}%)")

print("=" * 80)

# ============================================
# 第六部分：設定參數（⭐ 記憶體優化）
# ============================================

BASE_MODEL_ID = "google/medgemma-4b-it"

# ⭐ 記憶體優化的訓練參數
LEARNING_RATE = 1e-4
NUM_EPOCHS = 3
BATCH_SIZE = 2  # ⭐ 從 4 降到 2（減少記憶體使用）
MAX_LENGTH = 384  # ⭐ 從 512 降到 384（減少記憶體使用）
GRADIENT_ACCUMULATION_STEPS = 8  # ⭐ 從 4 增加到 8（保持有效 batch size = 16）

print(f"\n⚙️ 訓練參數設定（記憶體優化版）")
print("=" * 50)
print(f"使用模型: {BASE_MODEL_ID}")
print(f"學習率: {LEARNING_RATE}")
print(f"訓練輪數: {NUM_EPOCHS}")
print(f"批次大小: {BATCH_SIZE} ⭐ (優化)")
print(f"梯度累積步數: {GRADIENT_ACCUMULATION_STEPS} ⭐ (優化)")
print(f"有效批次大小: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"最大序列長度: {MAX_LENGTH} ⭐ (優化)")
print("=" * 50)

# ============================================
# 第七部分：建立訓練格式
# ============================================

def create_training_example(row):
    """
    將每一筆資料轉換成模型訓練用的格式
    """
    term = row['Term_To_Check']
    output = row['Correct_Output']

    # 建立 prompt
    prompt = f"Please check if there is any error in this medical term: {term}"

    # 使用 medgemma 的對話格式
    formatted_prompt = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n{output}<end_of_turn>"

    return {
        'text': formatted_prompt,
        'term': term,
        'response': output,
        'category': row['category']
    }

# 轉換所有資料
print("\n🔄 正在建立訓練格式...")
train_data = [create_training_example(row) for _, row in df.iterrows()]

print(f"✅ 訓練格式建立完成")
print(f"  總樣本數: {len(train_data)}")

# ⭐ 釋放記憶體
del df
gc.collect()

# 顯示範例
print(f"\n📝 訓練資料範例（前2筆）:")
for i, example in enumerate(train_data[:2]):
    print(f"\n範例 {i+1}:")
    print(f"  類別: {example['category']}")
    print(f"  術語: {example['term']}")
    print(f"  回應: {example['response'][:80]}...")

# ============================================
# 第八部分：智能資料分割
# ============================================

print("\n" + "=" * 80)
print("🔀 資料分割")
print("=" * 80)

from sklearn.model_selection import train_test_split

# 分離正確和錯誤案例
correct_examples = [ex for ex in train_data if ex['response'] == 'No issues found.']
error_examples = [ex for ex in train_data if ex['response'] != 'No issues found.']

print(f"\n資料統計:")
print(f"  正確案例: {len(correct_examples)}")
print(f"  錯誤案例: {len(error_examples)}")

# 對正確和錯誤案例分別進行 stratified split
correct_train, correct_test = train_test_split(
    correct_examples,
    test_size=0.1,
    random_state=42
)

error_train, error_test = train_test_split(
    error_examples,
    test_size=0.1,
    random_state=42
)

# 合併訓練集和測試集
train_examples = correct_train + error_train
test_examples = correct_test + error_test

# 打亂順序
import random
random.seed(42)
random.shuffle(train_examples)
random.shuffle(test_examples)

# 計算統計資訊
train_error_count = sum(1 for ex in train_examples if ex['response'] != 'No issues found.')
test_error_count = sum(1 for ex in test_examples if ex['response'] != 'No issues found.')

print(f"\n✅ 資料分割完成")
print(f"\n訓練集:")
print(f"  總數: {len(train_examples)}")
print(f"  正確: {len(train_examples) - train_error_count} ({(len(train_examples) - train_error_count)/len(train_examples)*100:.1f}%)")
print(f"  錯誤: {train_error_count} ({train_error_count/len(train_examples)*100:.1f}%)")

print(f"\n測試集:")
print(f"  總數: {len(test_examples)}")
print(f"  正確: {len(test_examples) - test_error_count} ({(len(test_examples) - test_error_count)/len(test_examples)*100:.1f}%)")
print(f"  錯誤: {test_error_count} ({test_error_count/len(test_examples)*100:.1f}%)")

print("=" * 80)

# ⭐ 釋放記憶體
del correct_examples, error_examples, correct_train, correct_test, error_train, error_test
gc.collect()

# ============================================
# 第九部分：載入模型和分詞器
# ============================================

print("\n📥 正在載入模型和分詞器...")

# 載入分詞器
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# 設定 padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✅ 分詞器載入完成")
print(f"  Vocabulary 大小: {len(tokenizer)}")

# ⭐ 記憶體優化的量化配置
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 載入模型
print(f"\n正在載入模型（使用 4-bit 量化）...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

print(f"✅ 模型載入完成")
print(f"  模型類型: {type(model).__name__}")

# ⭐ 清理 GPU 快取
torch.cuda.empty_cache()
gc.collect()

# ============================================
# 第十部分：配置 LoRA
# ============================================

print("\n🔧 配置 LoRA...")

# 準備模型進行 k-bit 訓練
model = prepare_model_for_kbit_training(model)

# LoRA 設定
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    task_type="CAUSAL_LM",
    bias="none"
)

# 應用 LoRA
model = get_peft_model(model, lora_config)

print("✅ LoRA 配置完成")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")

# 顯示可訓練參數
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n參數統計:")
print(f"  可訓練參數: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"  總參數: {total_params:,}")

# ============================================
# 第十一部分：準備訓練資料
# ============================================

print("\n🔄 準備訓練資料...")

def tokenize_function(examples):
    """
    將文本資料轉換成模型輸入格式
    """
    # 取出文本
    texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]

    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors=None
    )

    # 設定 labels
    tokenized['labels'] = tokenized['input_ids'].copy()

    return tokenized

# 轉換成 Dataset 格式
train_dataset_dict = {
    'text': [ex['text'] for ex in train_examples],
    'term': [ex['term'] for ex in train_examples],
    'response': [ex['response'] for ex in train_examples],
    'category': [ex['category'] for ex in train_examples]
}

test_dataset_dict = {
    'text': [ex['text'] for ex in test_examples],
    'term': [ex['term'] for ex in test_examples],
    'response': [ex['response'] for ex in test_examples],
    'category': [ex['category'] for ex in test_examples]
}

train_dataset = Dataset.from_dict(train_dataset_dict)
test_dataset = Dataset.from_dict(test_dataset_dict)

# ⭐ 釋放記憶體
del train_dataset_dict, test_dataset_dict
gc.collect()

# Tokenize
print("  Tokenizing 訓練集...")
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text', 'term', 'response', 'category'],
    batch_size=100  # ⭐ 分批處理避免記憶體問題
)

print("  Tokenizing 測試集...")
test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text', 'term', 'response', 'category'],
    batch_size=100
)

print(f"✅ 訓練資料準備完成")
print(f"  訓練集大小: {len(train_dataset)}")
print(f"  測試集大小: {len(test_dataset)}")

# ⭐ 清理 GPU 快取
torch.cuda.empty_cache()
gc.collect()

# ============================================
# 第十二部分：設定訓練參數
# ============================================

print("\n" + "=" * 80)
print("⚙️ 設定訓練參數")
print("=" * 80)

# 設定輸出目錄
output_dir = "/content/drive/MyDrive/medgemma_balanced_oom_fixed_2"

# 確保目錄存在
os.makedirs(output_dir, exist_ok=True)
print(f"📁 輸出目錄: {output_dir}")

# 訓練參數
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,

    # 梯度累積
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,

    # 學習率設定
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,

    # 評估策略
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,  # ⭐ 從 3 降到 2（節省空間）

    # 早停
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # 日誌設定
    logging_steps=20,
    logging_dir=f"{output_dir}/logs",
    logging_first_step=True,

    # 其他設定
    push_to_hub=False,
    report_to="none",
    fp16=True,
    remove_unused_columns=False,

    # Seed
    seed=42,
    data_seed=42,

    # 優化設定
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    max_grad_norm=1.0,

    # ⭐ 記憶體優化
    gradient_checkpointing=True,  # 啟用梯度檢查點
    dataloader_num_workers=0,  # 避免多進程記憶體問題
)

print("✅ 訓練參數設定完成")
print(f"\n關鍵參數:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size} ⭐")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps} ⭐")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Max length: {MAX_LENGTH} ⭐")
print(f"  Gradient checkpointing: {training_args.gradient_checkpointing} ⭐")

# ============================================
# 第十三部分：開始訓練
# ============================================

print("\n" + "=" * 80)
print("🚀 準備開始訓練")
print("=" * 80)

# 創建 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

print("\n✅ Trainer 創建完成")
print(f"  訓練樣本數: {len(train_dataset)}")
print(f"  評估樣本數: {len(test_dataset)}")
print(f"  預計訓練步數: {len(train_dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS) * NUM_EPOCHS}")

# ⭐ 最後一次清理 GPU 快取
torch.cuda.empty_cache()
gc.collect()

print("\n" + "=" * 80)
print("🏃 開始訓練...")
print("=" * 80)
print("提示：")
print("  - 使用記憶體優化設定（batch_size=2, max_length=384）")
print("  - 啟用梯度檢查點節省記憶體")
print("  - 每 100 steps 會進行一次評估")
print("  - 如果 eval_loss 不再下降，會自動早停")
print("=" * 80)

# 開始訓練
try:
    train_result = trainer.train()

    print("\n" + "=" * 80)
    print("✅ 訓練完成！")
    print("=" * 80)
    print(f"訓練統計:")
    print(f"  最終 train_loss: {train_result.training_loss:.4f}")
    print(f"  訓練時間: {train_result.metrics['train_runtime']:.2f} 秒")

    # 保存模型
    print(f"\n💾 保存模型到: {output_dir}/final_model")
    trainer.save_model(f"{output_dir}/final_model")
    tokenizer.save_pretrained(f"{output_dir}/final_model")
    print("✅ 模型保存完成")

except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n" + "=" * 80)
        print("❌ 仍然出現 Out of Memory 錯誤")
        print("=" * 80)
        print("建議進一步降低參數：")
        print("  1. BATCH_SIZE = 1")
        print("  2. MAX_LENGTH = 256")
        print("  3. GRADIENT_ACCUMULATION_STEPS = 16")
        print("=" * 80)
    raise e

# ============================================
# 訓練後測試
# ============================================

print("\n" + "=" * 80)
print("🧪 訓練後測試")
print("=" * 80)

# 測試已知的錯誤術語
test_cases = [
    ("acne valgaris", "應該是 acne vulgaris"),
    ("Tegratol Allergy", "應該是 Tegretol Allergy"),
    ("Fattu Liver", "應該是 Fatty Liver"),
    ("Memetasone", "應該是 Memotasone"),
    ("Other anomalites", "應該是 anomalies"),
]

print("\n測試已知的錯誤術語:")
print("-" * 80)

model.eval()
success_count = 0

for i, (term, expected) in enumerate(test_cases, 1):
    prompt = f"Please check if there is any error in this medical term: {term}"
    formatted = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True
        )

    result = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    result = result.strip()

    is_correct = ("incorrect" in result.lower() and "should be" in result.lower())
    status = "✅ 正確識別" if is_correct else "❌ 識別失敗"

    print(f"\n{i}. 術語: {term}")
    print(f"   預期: {expected}")
    print(f"   輸出: {result}")
    print(f"   狀態: {status}")

    if is_correct:
        success_count += 1

print("\n" + "-" * 80)
print(f"測試結果: {success_count}/{len(test_cases)} ({success_count/len(test_cases)*100:.1f}%)")

if success_count == len(test_cases):
    print("🎉 所有測試通過！模型訓練成功！")
elif success_count >= len(test_cases) * 0.8:
    print("✅ 大部分測試通過，模型表現良好")
else:
    print("⚠️ 部分測試失敗，可能需要調整參數重新訓練")

print("=" * 80)

# ============================================
# 完整的 F1 Score 評估
# ============================================

print("\n" + "=" * 80)
print("📊 完整評估（F1 Score）")
print("=" * 80)

print("\n🔄 正在評估測試集...")
print(f"提示：評估 {len(test_examples)} 個樣本，可能需要幾分鐘...")

model.eval()
predictions = []
ground_truths = []

# 對測試集進行預測
for i, example in enumerate(test_examples):
    if i % 20 == 0:
        print(f"  評估進度: {i}/{len(test_examples)}")

    term = example['term']
    true_response = example['response']

    prompt = f"Please check if there is any error in this medical term: {term}"
    formatted = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True
        )

    pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()

    # 判斷預測類別
    pred_label = 1 if ("incorrect" in pred_text.lower() or "error" in pred_text.lower()) else 0
    true_label = 0 if true_response == "No issues found." else 1

    predictions.append(pred_label)
    ground_truths.append(true_label)

# 計算指標
accuracy = accuracy_score(ground_truths, predictions)
precision = precision_score(ground_truths, predictions, zero_division=0)
recall = recall_score(ground_truths, predictions, zero_division=0)
f1 = f1_score(ground_truths, predictions, zero_division=0)

print("\n" + "=" * 80)
print("📈 評估結果")
print("=" * 80)
print(f"\n測試集大小: {len(test_examples)}")

print(f"\n評估指標:")
print(f"  ✅ Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  📊 Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"  🎯 Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"  ⭐ F1 Score:  {f1:.4f} ({f1*100:.2f}%)")

# 混淆矩陣
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ground_truths, predictions)

print(f"\n混淆矩陣:")
print(f"              預測正確  預測錯誤")
print(f"  實際正確      {cm[0,0]:4d}      {cm[0,1]:4d}")
print(f"  實際錯誤      {cm[1,0]:4d}      {cm[1,1]:4d}")

# 判斷模型表現
if f1 >= 0.9:
    print("\n🎉 模型表現優秀！F1 Score >= 0.9")
elif f1 >= 0.8:
    print("\n✅ 模型表現良好！F1 Score >= 0.8")
elif f1 >= 0.7:
    print("\n⚠️ 模型表現尚可，建議進一步優化")
else:
    print("\n❌ 模型表現不佳，建議調整參數重新訓練")

print("=" * 80)

# ============================================
# 最終摘要
# ============================================

print("\n" + "=" * 80)
print("📋 訓練完成摘要")
print("=" * 80)

print(f"\n✅ 記憶體優化設定:")
print(f"  - Batch size: {BATCH_SIZE} (優化)")
print(f"  - Max length: {MAX_LENGTH} (優化)")
print(f"  - Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"  - Gradient checkpointing: 啟用")
print(f"  - 最大樣本數: {MAX_TOTAL_SAMPLES}")

print(f"\n✅ 資料統計:")
print(f"  - 訓練集: {len(train_examples)} 筆")
print(f"  - 測試集: {len(test_examples)} 筆")

print(f"\n✅ 評估結果:")
print(f"  - 訓練後測試: {success_count}/{len(test_cases)} 通過")
print(f"  - F1 Score: {f1:.4f}")
print(f"  - Accuracy: {accuracy:.4f}")

print(f"\n✅ 模型位置:")
print(f"  - {output_dir}/final_model")

print("\n" + "=" * 80)
print("🎉 所有任務完成！")
print("=" * 80)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 43.3 MB/s eta 0:00:00
📁 正在掛載 Google Drive...
Mounted at /content/drive
✅ Google Drive 掛載完成！

📤 請上傳您的 Excel 訓練檔案
提示：程式會自動讀取 Excel 中的所有工作表
  - 支援多個工作表 (例如: MH, CM, AE)
  - 自動合併所有工作表的資料


Saving Training data 1028.xlsx to Training data 1028.xlsx
✅ 已上傳檔案: Training data 1028.xlsx

📊 正在讀取 Excel 檔案...
✅ 發現 3 個工作表: ['MH', 'CM', 'AE']
  📄 MH: 543 筆 (錯誤: 100 / 18.4%)
  📄 CM: 530 筆 (錯誤: 146 / 27.5%)
  📄 AE: 247 筆 (錯誤: 25 / 10.1%)

✅ 資料合併完成
  總筆數: 1320
  正確術語: 1049 筆
  錯誤術語: 271 筆

📊 資料平衡處理（過採樣 + 記憶體優化）

📈 原始資料分布:
  總筆數: 1320
  正確術語: 1049 (79.5%)
  錯誤術語: 271 (20.5%)

🎯 目標錯誤比例: 40.0%
🎯 最大樣本數: 3000

⚠️ 錯誤比例不足 (20.5% < 40.0%)
🔄 開始過採樣...

📊 計算結果:
  原始錯誤案例: 271
  需要錯誤案例: 699
  需要增加: 428

✅ 過採樣完成
  過採樣後錯誤案例: 699

🎉 資料平衡完成！
  總筆數: 1748
  正確術語: 1049 (60.0%)
  錯誤術語: 699 (40.0%)

⚙️ 訓練參數設定（記憶體優化版）
使用模型: google/medgemma-4b-it
學習率: 0.0001
訓練輪數: 3
批次大小: 2 ⭐ (優化)
梯度累積步數: 8 ⭐ (優化)
有效批次大小: 16
最大序列長度: 384 ⭐ (優化)

🔄 正在建立訓練格式...
✅ 訓練格式建立完成
  總樣本數: 1748

📝 訓練資料範例（前2筆）:

範例 1:
  類別: CM
  術語: Amoxycillin
  回應: Incorrect drug name: [Amoxycillin] should be [Amoxicillin]....

範例 2:
  類別: CM
  術語: Amoxycillin
  回應: Incorrect drug name: [Amoxycillin] should be [Amoxicillin]....

🔀 資料分割

資料統計:
  正確案例: 1049

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

...

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

✅ 訓練資料準備完成
  訓練集大小: 1573
  測試集大小: 175

⚙️ 設定訓練參數
📁 輸出目錄: /content/drive/MyDrive/medgemma_balanced_oom_fixed_2
✅ 訓練參數設定完成

關鍵參數:
  Epochs: 3
  Batch size: 2 ⭐
  Gradient accumulation: 8 ⭐
  Effective batch size: 16
  Max length: 384 ⭐
  Gradient checkpointing: True ⭐

🚀 準備開始訓練

✅ Trainer 創建完成
  訓練樣本數: 1573
  評估樣本數: 175
  預計訓練步數: 294

🏃 開始訓練...
提示：
  - 使用記憶體優化設定（batch_size=2, max_length=384）
  - 啟用梯度檢查點節省記憶體
  - 每 100 steps 會進行一次評估
  - 如果 eval_loss 不再下降，會自動早停


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,0.272800,0.298725
200,0.190600,0.249603



✅ 訓練完成！
訓練統計:
  最終 train_loss: 0.7819
  訓練時間: 1854.16 秒

💾 保存模型到: /content/drive/MyDrive/medgemma_balanced_oom_fixed_2/final_model
✅ 模型保存完成

🧪 訓練後測試

測試已知的錯誤術語:
--------------------------------------------------------------------------------

1. 術語: acne valgaris
   預期: 應該是 acne vulgaris
   輸出: Incorrect diagnosis name: [acne valgaris] should be [acne vulgaris].
   狀態: ✅ 正確識別

2. 術語: Tegratol Allergy
   預期: 應該是 Tegretol Allergy
   輸出: Incorrect diagnosis name: [Tegratol Allergy] should be [Tegretol Allergy].model
parent
   狀態: ✅ 正確識別

3. 術語: Fattu Liver
   預期: 應該是 Fatty Liver
   輸出: Incorrect event name: [Fattu Liver] should be [Fatty Liver].
   狀態: ✅ 正確識別

4. 術語: Memetasone
   預期: 應該是 Memotasone
   輸出: Incorrect drug name:[Memetasone] should be [Memetasone furoate cream].model
color_error_name:[Memetasone furoate cream]
   狀態: ✅ 正確識別

5. 術語: Other anomalites
   預期: 應該是 anomalies
   輸出: Incorrect diagnosis name: [Other anomalites] should be [Other anomalies].model
cardio_image_diagnos